In [22]:
from bokeh.colors import RGB
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, ColumnDataSource
from IPython import display
import pandas as pd
from openpyxl.workbook import Workbook
import time
from datetime import datetime

api_key = "https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CNY&apikey=RDZ3Q4TD6ELVMRJJ&datatype=csv"
btc_df=pd.read_csv(api_key,parse_dates=["timestamp"])
### to change symbol, see digital_currency_list.csv
### to change comparing market, see physical_currency_list.csv


#btc_daily_high = btc_df[["timestamp", "close (USD)"]]

pd.options.display.float_format = '{:.1f}'.format

#print(btc_df)
#show(p)
# writer = pd.ExcelWriter('output.xlsx')
# btc_df.to_excel(writer,'Sheet1')
# writer.save()

In [23]:
def preprocess_features(btc_df):
  
  selected_features = btc_df[[
      "open (USD)",
      "high (USD)",
      "low (USD)",  
      "volume"]]
  
  processed_features = selected_features.copy()
  processed_features["open_close_diff"] =  btc_df["close (USD)"] - btc_df["open (USD)"]
  
  return processed_features

def preprocess_targets(btc_df):
  
  output_targets = btc_df[["close (USD)"]]
  return output_targets

In [24]:
# training_examples = preprocess_features(btc_df_r.head(1100))
# training_targets = preprocess_targets(btc_df_r.head(1100))
# validation_examples = preprocess_features(btc_df_r.tail(560))
# validation_targets = preprocess_targets(btc_df_r.tail(560))
all_examples = preprocess_features(btc_df)

btc_df_c = btc_df.copy()
btc_df_c["open_close_diff"] = all_examples["open_close_diff"]

In [42]:
btc_df_c["date_string"] = btc_df_c["timestamp"].dt.strftime("%Y-%m-%d")
btc_df_c["oc_diff_string"] = btc_df_c["open_close_diff"]
btc_df_c["close_string"] = btc_df_c["close (USD)"]
#print(btc_df_c["date_string"])

cds=ColumnDataSource(btc_df_c)

#print(cds)

#df1 = pd.DataFrame(btc_df_c["timestamp"])
#print(df1)

p = figure(width=1000, height=900, x_axis_type="datetime", logo=None, background_fill_color=RGB(15, 15, 8))

p.title.text = "BTC 2014-Present"
p.title.text_color = "Orange"
p.title.text_font = "times"
p.title.text_font_style = "italic"
p.title.text_font_size = "20pt"

p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "USD"
#p.xgrid.grid_line_color = "#dddddd"


p.line("timestamp","open_close_diff", legend="Open to Close Diff", color="red",source=cds)
p.line("timestamp","close (USD)", legend="Close Price", color=RGB(114, 232, 51),source=cds)

p.legend.label_text_font = "times"
p.legend.label_text_font_style = "italic"
p.legend.label_text_color = "navy"
p.legend.location = "top_left"
p.legend.background_fill_color = "darkgray"

hover=HoverTool(tooltips=[("Date","@date_string"), ("Open to Close Diff","@oc_diff_string{0}"), ("Close","@close_string{0}")])
p.add_tools(hover)


output_file("BTC.html")

show(p)

In [26]:
btc_df_c["open_close_diff"].describe()

count    1661.0
mean        3.8
std       333.8
min     -5387.4
25%        -7.5
50%         0.8
75%        12.6
max      4497.9
Name: open_close_diff, dtype: float64

In [27]:
btc_df_c["close (USD)"].describe()

count    1661.0
mean     2605.7
std      3717.2
min       173.6
25%       381.2
50%       613.8
75%      3796.0
max     21794.6
Name: close (USD), dtype: float64